# Cracking Open the Hugging Face Transformers Library

Setup

In [26]:
from transformers import pipeline, Conversation
import gradio as gr

Sentiment Analysis

In [27]:
# toy example 1
pipeline(task="sentiment-analysis")("Love this!")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998745918273926}]

In [28]:
# toy example 2
pipeline(task="sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")("Love this!")

[{'label': 'POSITIVE', 'score': 0.9998745918273926}]

More Sentiment Analysis

In [29]:
# defining classifier
classifier = pipeline(task="sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

In [30]:
classifier("Hate this.")

[{'label': 'NEGATIVE', 'score': 0.9997110962867737}]

In [31]:
# we can also pass in a list to classifier
text_list = ["This is great", \
             "Thanks for nothing", \
             "You've got to work on your face", \
             "You're beautiful, never change!"]

classifier(text_list)

[{'label': 'POSITIVE', 'score': 0.9998785257339478},
 {'label': 'POSITIVE', 'score': 0.9680057168006897},
 {'label': 'NEGATIVE', 'score': 0.8776107430458069},
 {'label': 'POSITIVE', 'score': 0.9998120665550232}]

In [32]:
# if there are multiple target labels, we can return them all
classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

In [33]:
classifier(text_list[0])

[[{'label': 'admiration', 'score': 0.9526104927062988},
  {'label': 'approval', 'score': 0.03047209605574608},
  {'label': 'neutral', 'score': 0.015236238949000835},
  {'label': 'excitement', 'score': 0.006063771899789572},
  {'label': 'gratitude', 'score': 0.005296194460242987},
  {'label': 'joy', 'score': 0.004475211258977652},
  {'label': 'curiosity', 'score': 0.004322331864386797},
  {'label': 'realization', 'score': 0.0040895999409258366},
  {'label': 'optimism', 'score': 0.00407722033560276},
  {'label': 'disapproval', 'score': 0.004076569341123104},
  {'label': 'annoyance', 'score': 0.0035287411883473396},
  {'label': 'surprise', 'score': 0.0029730673413723707},
  {'label': 'disappointment', 'score': 0.002734639449045062},
  {'label': 'love', 'score': 0.00269458070397377},
  {'label': 'amusement', 'score': 0.0024867451284080744},
  {'label': 'confusion', 'score': 0.002360743237659335},
  {'label': 'pride', 'score': 0.0021013403311371803},
  {'label': 'sadness', 'score': 0.001773

Summarization

In [34]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [35]:
text = """
Hugging Face is an AI company that has become a major hub for open-source machine learning. 
Their platform has 3 major elements which allow users to access and share machine learning resources. 
First, is their rapidly growing repository of pre-trained open-source machine learning models for things such as natural language processing (NLP), computer vision, and more. 
Second, is their library of datasets for training machine learning models for almost any task. 
Third, and finally, is Spaces which is a collection of open-source ML apps.

The power of these resources is that they are community generated, which leverages all the benefits of open source i.e. cost-free, wide diversity of tools, high quality resources, and rapid pace of innovation. 
While these make building powerful ML projects more accessible than before, there is another key element of the Hugging Face ecosystem—their Transformers library.
"""
summarized_text = summarizer(text, min_length=5, max_length=140)[0]['summary_text']
summarized_text

'Hugging Face is an AI company that has become a major hub for open-source machine learning. They have 3 major elements which allow users to access and share machine learning resources.'

In [36]:
classifier(summarized_text)

[[{'label': 'neutral', 'score': 0.9101783633232117},
  {'label': 'approval', 'score': 0.08781367540359497},
  {'label': 'realization', 'score': 0.023256294429302216},
  {'label': 'annoyance', 'score': 0.006623796187341213},
  {'label': 'admiration', 'score': 0.004981078673154116},
  {'label': 'disapproval', 'score': 0.004730122163891792},
  {'label': 'optimism', 'score': 0.0033590728417038918},
  {'label': 'disappointment', 'score': 0.0026190034113824368},
  {'label': 'confusion', 'score': 0.0019539783243089914},
  {'label': 'excitement', 'score': 0.0012417081743478775},
  {'label': 'disgust', 'score': 0.0011407802812755108},
  {'label': 'joy', 'score': 0.0010540130315348506},
  {'label': 'amusement', 'score': 0.0009572383714839816},
  {'label': 'love', 'score': 0.0008871073368936777},
  {'label': 'desire', 'score': 0.0008553274674341083},
  {'label': 'curiosity', 'score': 0.0008261064067482948},
  {'label': 'anger', 'score': 0.0007336385897360742},
  {'label': 'caring', 'score': 0.000

Conversational

In [37]:
chatbot = pipeline("conversational", model="facebook/blenderbot-400M-distill")

In [38]:
conversation = Conversation("Hi I'm Shaw, how are you?")
conversation = chatbot(conversation)

In [39]:
conversation

Conversation id: d6ab5e13-1a09-463b-858e-ce5e21b6b9d4
user: Hi I'm Shaw, how are you?
assistant:  I'm doing well. How are you doing this evening? I just got home from work.

In [40]:
conversation.add_user_input("Where do you work?")
conversation = chatbot(conversation)

In [41]:
conversation

Conversation id: d6ab5e13-1a09-463b-858e-ce5e21b6b9d4
user: Hi I'm Shaw, how are you?
assistant:  I'm doing well. How are you doing this evening? I just got home from work.
user: Where do you work?
assistant:  I work at a grocery store. What about you? What do you do for a living?

Deploy Chatbot UI

Text Sentiment Chatbot

In [42]:
def top3_text_classes(message, history):
    return str(classifier(message)[0][:3]).replace('}, {', '\n').replace('[{', '').replace('}]', '')

demo_sentiment = gr.ChatInterface(top3_text_classes, title="Text Sentiment Chatbot", description="Enter your text, and the chatbot will classify the sentiment.")

demo_sentiment.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Summarizer Chatbot

In [43]:
def summarizer_bot(message, history):
    return summarizer(message, min_length=5, max_length=140)[0]['summary_text']

demo_summarizer = gr.ChatInterface(summarizer_bot, title="Summarizer Chatbot", description="Enter your text, and the chatbot will return the summarized version.")

demo_summarizer.launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


Vanilla Chatbot

In [44]:
message_list = []
response_list = []

def vanilla_chatbot(message, history):
    conversation = Conversation(text=message, past_user_inputs=message_list, generated_responses=response_list)
    conversation = chatbot(conversation)

    return conversation.generated_responses[-1]

demo_chatbot = gr.ChatInterface(vanilla_chatbot, title="Vanilla Chatbot", description="Enter text to start chatting.")

demo_chatbot.launch()

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
